In [6]:
#Dependencies
import requests
import json
from config import ow_api
import matplotlib.pyplot as plt 
from citipy import citipy
import pandas as pd 
import numpy as np 

In [37]:
#Get random coordinates
rand_lat = np.random.uniform(-90,high=90,size=(1500,1))
rand_lon = np.random.uniform(-180,high=180,size=(1500,1))
#print(rand_lat)

In [38]:
#Get list of random cities
cities = [citipy.nearest_city(rand_lat[i],rand_lon[i]) for i in range(len(rand_lat))]
city_names = [cities[i].city_name for i in range(len(cities))]
#print(city_names)

In [39]:
#Dedupe cities and make sure there are more than 500
dedupe_cities = list(dict.fromkeys(city_names))
print(len(dedupe_cities))

589


In [47]:
#Use first city to determine json structure
test_url = f"http://api.openweathermap.org/data/2.5/weather?q={dedupe_cities[0]}&appid={ow_api}&units=imperial"
response = requests.get(test_url).json()
print(json.dumps(response,indent=4,sort_keys=True))

{
    "base": "stations",
    "clouds": {
        "all": 75
    },
    "cod": 200,
    "coord": {
        "lat": 62.46,
        "lon": -114.35
    },
    "dt": 1599338646,
    "id": 6185377,
    "main": {
        "feels_like": 43.5,
        "humidity": 57,
        "pressure": 1019,
        "temp": 51.48,
        "temp_max": 51.8,
        "temp_min": 51.01
    },
    "name": "Yellowknife",
    "sys": {
        "country": "CA",
        "id": 1007,
        "sunrise": 1599309478,
        "sunset": 1599359652,
        "type": 1
    },
    "timezone": -21600,
    "visibility": 10000,
    "weather": [
        {
            "description": "broken clouds",
            "icon": "04d",
            "id": 803,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 30,
        "speed": 9.17
    }
}


In [45]:
#Create dataframe to hold information
citydata_df = pd.DataFrame({"Cities":dedupe_cities})

#Add columns to dataframe
citydata_df["Lat"] = ""
citydata_df["Lon"] = ""
citydata_df["Temp (F)"] = ""
citydata_df["Humidity (%)"] = ""
citydata_df["Cloudiness (%)"] = ""
citydata_df["Wind Speed (mph)"] = ""

citydata_df.head()

,Cities,Lat,Lon,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,yellowknife,,,,,,
1,carnarvon,,,,,,
2,namibe,,,,,,
3,hobart,,,,,,
4,hermanus,,,,,,


In [59]:
base_url = f"http://api.openweathermap.org/data/2.5/weather?q="
api_key = f"&appid={ow_api}&units=imperial"

for index, row in citydata_df.iterrows():

    query_city = row["Cities"]
    
    query_url = f"{base_url} + {query_city} + {api_key}"

    print(f"Retrieving results for Index {index}: {query_city}...")
    response = requests.get(query_url).json()

    #results = response["results"]

    try:
        print(f"Found city {response['name']}, City ID No.: {response['id']}")

        citydata_df.loc[index, "Lat"] = response["coord"]["lat"]
        citydata_df.loc[index, "Lon"] = response["coord"]["lon"]
        citydata_df.loc[index, "Temp (F)"] = response["main"]["temp"]
        citydata_df.loc[index, "Humidity (%)"] = response["main"]["humidity"]
        citydata_df.loc[index, "Cloudiness (%)"] = response["clouds"]["all"]
        citydata_df.loc[index, "Wind Speed (mph)"] = response["wind"]["speed"]
    except (KeyError,IndexError):
        print(f"Missing field/result ... skipping ...")
    
    print("-------------")

..
Found city Nigde, City ID No.: 303826
-------------
Retrieving results for Index 393: port blair...
Found city Port Blair, City ID No.: 1259385
-------------
Retrieving results for Index 394: gat...
Missing field/result ... skipping ...
-------------
Retrieving results for Index 395: ballina...
Found city Ballina, City ID No.: 2177069
-------------
Retrieving results for Index 396: pudozh...
Found city Pudozh, City ID No.: 504269
-------------
Retrieving results for Index 397: delta del tigre...
Found city Delta del Tigre, City ID No.: 3442778
-------------
Retrieving results for Index 398: ozark...
Found city Ozark, City ID No.: 4402245
-------------
Retrieving results for Index 399: sale...
Found city Sale, City ID No.: 2537763
-------------
Retrieving results for Index 400: saint-georges...
Found city Saint-Georges, City ID No.: 6138282
-------------
Retrieving results for Index 401: beaufort...
Found city Beaufort, City ID No.: 1733874
-------------
Retrieving results for Index 

In [60]:
citydata_df.head()

,Cities,Lat,Lon,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,yellowknife,62.46,-114.35,51.46,57,75,9.17
1,carnarvon,-24.87,113.63,66.2,93,90,14.99
2,namibe,-15.2,12.15,64.09,77,1,5.84
3,hobart,-42.88,147.33,46.22,81,75,9.17
4,hermanus,-34.42,19.23,52,95,97,7.34
